##Setup



In [1]:
!pip install -q pytorch-lightning
!pip install -q transformers
!pip install -q gpt-2-simple
!pip install rake-nltk
!pip install tensorflow==1.15.3

     |████████████████████████████████| 829kB 7.6MB/s 
     |████████████████████████████████| 112kB 34.9MB/s 
     |████████████████████████████████| 829kB 24.7MB/s 
     |████████████████████████████████| 276kB 46.7MB/s 
     |████████████████████████████████| 1.3MB 46.2MB/s 
     |████████████████████████████████| 296kB 48.0MB/s 
     |████████████████████████████████| 143kB 56.2MB/s 
     |████████████████████████████████| 1.9MB 13.0MB/s 
     |████████████████████████████████| 890kB 54.2MB/s 
     |████████████████████████████████| 3.2MB 48.5MB/s 
     |████████████████████████████████| 110.5MB 99kB/s 
     |████████████████████████████████| 3.8MB 43.9MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
     |████████████████████████████████| 512kB 53.3MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=37de794bc31c40dc64fa4169819b2f2e9dee472850bf0814a301adf8a6758146
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f

In [2]:
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

import math
import random
import re
import argparse

import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
gpt2.download_gpt2(model_name="117M")

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 287Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 1.07Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 425Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:29, 16.7Mit/s]
Fetching model.ckpt.index: 1.05Mit [00:00, 486Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 1.60Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 1.33Mit/s]


##BART Part

In [3]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'BART/'

Mounted at /content/gdrive


In [4]:
class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model, hparams):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    self.hparams = hparams

    if self.hparams.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.hparams.freeze_embeds:
      self.freeze_embeds()
  
  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))
    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)
    
    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss': val_loss}
  
  # Method that generates text using the BartForConditionalGeneration's generate() method
  def generate_text(self, text, eval_beams, early_stopping = True, max_len = 40, startT = None):
    ''' Function to generate text '''
    if startT == None:
      dstartT = self.tokenizer.pad_token_id
    else:
      dstartT = tokenizer.convert_tokens_to_ids(startT)
    generated_ids = self.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = dstartT,
        num_beams= eval_beams,
        max_length = max_len,
        early_stopping = early_stopping
    )
    return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]

def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False


In [5]:
# Create a dataloading module as per the PyTorch Lightning Docs
class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, data_file, batch_size, num_examples = 20000):
    super().__init__()
    self.tokenizer = tokenizer
    self.data_file = data_file
    self.batch_size = batch_size
    self.num_examples = num_examples
  
  # Loads and splits the data into training, validation and test sets with a 60/20/20 split
  def prepare_data(self):
    self.data = pd.read_csv(self.data_file)[:self.num_examples]
    self.train, self.validate, self.test = np.split(self.data.sample(frac=1), [int(.6*len(self.data)), int(.8*len(self.data))])

  # encode the sentences using the tokenizer  
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train['source'], self.train['target'])
    self.validate = encode_sentences(self.tokenizer, self.validate['source'], self.validate['target'])
    self.test = encode_sentences(self.tokenizer, self.test['source'], self.test['target'])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])                          
    train_data = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels']) 
    val_data = DataLoader(dataset, batch_size = self.batch_size)                       
    return val_data

  def test_dataloader(self):
    dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels']) 
    test_data = DataLoader(dataset, batch_size = self.batch_size)                   
    return test_data



In [6]:
hparams = argparse.Namespace()

hparams.freeze_encoder = True
hparams.freeze_embeds = True
hparams.eval_beams = 4

In [7]:
def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=32, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence 
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )
    # Shift the target ids to the right
    # shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)
  

  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch


def noise_sentence(sentence_, percent_words, replacement_token = "<mask>"):
  '''
  Function that noises a sentence by adding <mask> tokens
  Args: sentence - the sentence to noise
        percent_words - the percent of words to replace with <mask> tokens; the number is rounded up using math.ceil
  Returns a noised sentence
  '''
  # Create a list item and copy
  sentence_ = sentence_.split(' ')
  sentence = sentence_.copy()
  
  num_words = math.ceil(len(sentence) * percent_words)
  
  # Create an array of tokens to sample from; don't include the last word as an option because in the case of lyrics
  # that word is often a rhyming word and plays an important role in song construction
  sample_tokens = set(np.arange(0, np.maximum(1, len(sentence)-1)))
  
  words_to_noise = random.sample(sample_tokens, num_words)
  
  # Swap out words, but not full stops
  for pos in words_to_noise:
      if sentence[pos] != '.':
          sentence[pos] = replacement_token
  
  # Remove redundant spaces
  sentence = re.sub(r' {2,5}', ' ', ' '.join(sentence))
  
  # Combine concurrent <mask> tokens into a single token; this just does two rounds of this; more could be done
  sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
  sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
  return sentence
  

In [8]:
# Load the model
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained(
    "facebook/bart-base")


In [9]:
# Load the data into the model for training
summary_data = SummaryDataModule(tokenizer, '/content/gdrive/My Drive/BART learns to rap/lyrics_simple_noised.csv',
                                 batch_size = 16, num_examples = 140000)

# Load the model from a pre-saved checkpoint; alternatively use the code below to start training from scratch
# model = LitModel.load_from_checkpoint(base_dir + "checkpoint_files_2/8_ep_140k_simple_0210.ckpt",
#                                       learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

In [10]:
#checkpoint = ModelCheckpoint(filepath=base_dir + 'checkpoint_files_2/')
trainer = pl.Trainer(gpus = 1,
                     max_epochs = 2,
                     min_epochs = 2,
                     auto_lr_find = False,
                     
                     progress_bar_refresh_rate = 500)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [11]:
# Fit the instantiated model to the data
trainer.fit(model, summary_data)


  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


1

In [12]:
def generate_lyrics(seed_line, num_lines, model_, startW, noise_percent = 0.25, multiple_lines = False, max_line_history = 3):
  ''' Function that generates lyrics based on previously generated lyrics 
      Args: seed_line - a line to start off the machine
            num_lines - the number of lines to generate
            model_ - the model used to generate the text
            multiple_lines - whether the model generates based on multiple previous lines or just the past line
            max_line_history - the maximum number of previous lines used in the current input
      Returns a list with num_lines of rap lines
  '''
  # Put the model on eval mode
  model_.to(torch.device('cpu'))
  model_.eval()
  lyrics = []
  lyrics.append(seed_line)
  prompt_line_tokens = tokenizer(noise_sentence(seed_line, 0.2), max_length = 32, return_tensors = "pt", truncation = True)
  # Loop through the number of lines generating a new line based on the old

  line = [seed_line]
  for i in range(num_lines):
    # Print out the new line
    print(line[0].strip())
    lyrics.append(line[0])
    line = model.generate_text(prompt_line_tokens, eval_beams = 4, startT = startW)
    # This deals with an artefact in the training data that I had an issue cleaning
    if line[0].find(":") != -1:
      line[0] = re.sub(r'[A-Z]+: ', '', line[0])
    # This allows the model to generate a new line conditioned on more than one line
    if multiple_lines:
      start_line = np.maximum(0, i - max_line_history)
      end_line = i
      prompt_line = ' '.join(lyrics[start_line:end_line]) # Going to end_line is fine because it is non-inclusive
    else:
      prompt_line = lyrics[i]
    prompt_line_tokens = tokenizer(noise_sentence(prompt_line, noise_percent), max_length = 32, return_tensors = "pt", truncation = True)

  return lyrics

#GPT-2 Part

In [13]:
gpt2.mount_gdrive()
file_name = "reversed.txt"
gpt2.copy_file_from_gdrive(file_name)

Mounted at /content/drive


In [14]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [15]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-100
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-100


#Rhyming Part

In [16]:
import random, re
from google.colab import files
uploaded = files.upload()

Saving dict.txt to dict.txt


In [17]:

import pandas as pd

STRESSES = {'AA1', 'AE1', 'AH1', 'AO1', 'AW1', 'AY1', 'EH1', 'ER1', 'EY1', 'IH1', 'IY1', 'OW1', 'OY1', 'UH1', 'UW1'}
"""
def fileToDict(filepath):
    dictA = dict()
    with open(filepath) as fileA:
        for line in fileA:
            words = line.split() 
            dictA[words[0]] = tuple(words[1:])
    return dictA

PDICT = fileToDict("CMUdict.txt") 
print(len(PDICT.keys()))

import nltk 
nltk.download('words')
nltk.download('punkt')
file_content = open("fusion.txt").read()
tokens = nltk.word_tokenize(file_content)

def cleanDict (pDict):
  phodict = {} 
  for i in pDict.keys():
     if ((i.lower() in tokens) and (i.isalpha())):
       phodict[i] = pDict[i]
  return phodict
PHODICT = cleanDict (PDICT)
print(len(PHODICT.keys()))
"""
import ast
file = open("dict.txt", "r")

contents = file.read()
PHODICT = ast.literal_eval(contents)

file.close()

def isSubList(listA, listB):
    if len(listA) < len(listB):
        return isSubList(listB, listA)
    n = len(listB)
    for start in range(len(listA)-n+1):
        if all(listA[start+i] == listB[i] for i in range(n)):
            return True
    return False

def getRhymes(word):
    print(type(word))
    sounds = PHODICT[word]
    for index,sound in enumerate(reversed(sounds)):
        if sound in STRESSES:
            ending = sounds[-index-1:]
            break
    yielded = set()
    for wordB, soundsB in PHODICT.items():
        if (ending == soundsB[-index-1:]) and (soundsB not in yielded) and (not isSubList(sounds, soundsB)):
            yielded.add(soundsB)
            yield wordB

#Generation Part

In [18]:
def rhymeFinder(inWord):
  lastWord = inWord
  sameRhyme = []
  for word in getRhymes(lastWord.upper()):
    sameRhyme.append(word.lower())
  outWord = random.choice(sameRhyme)
  return outWord

def reverseSentence(inList):
  words = inList[0].split()
  outStr = ' '.join(reversed(words))
  return outStr

In [127]:
firstSeed = "Find some rhyme"
allLyrics = []

BART_sentence = generate_lyrics(seed_line = firstSeed, num_lines = 2, model_ = model, startW = None,
                           noise_percent = 0.25, multiple_lines = False, max_line_history = 1)

allLyrics.append(BART_sentence[-1])

startword = rhymeFinder(BART_sentence[-1].split()[-1])
GPT2_sentence = gpt2.generate(sess,
              temperature=0.7,
              prefix=startword,
              truncate ='\n',
              nsamples = 1,
              batch_size= 1,
              length = 30,
              return_as_list=True
              )
reversed_GPT2 = reverseSentence(GPT2_sentence)

allLyrics.append(reversed_GPT2)
print(allLyrics)

Find some rhyme
I don't need no rhyme
<class 'str'>
[" I don't need no rhyme", "And not that's a's a-hole dime"]


In [128]:
from rake_nltk import Rake

r = Rake()
r.extract_keywords_from_sentences(allLyrics)
keywords = r.get_ranked_phrases()

keySentence = ''
for i in keywords:
  keySentence += i
  keySentence += ' '

In [130]:

BART_sentence1 = generate_lyrics(seed_line = keySentence, num_lines = 2, model_ = model,startW= GPT2_sentence[0],
                           noise_percent = 0.25, multiple_lines = False, max_line_history = 1)

startword = rhymeFinder(BART_sentence1[-1].split()[-1])
GPT2_sentence1 = gpt2.generate(sess,
              temperature=0.7,
              prefix=startword,
              truncate ='\n',
              nsamples = 1,
              batch_size= 1,
              length = 30,
              return_as_list=True
              )
reversed_GPT21 = reverseSentence(GPT2_sentence1)
print(reversed_GPT21)

hole dime rhyme need
I ain't got no time for that
<class 'str'>
we need no, much to be, my much to be, your richest spend, to fat's
